In [1]:
#Web scraping
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

#data processing
import re
import nltk

#AWS
import boto3
from botocore.exceptions import ClientError
from io import StringIO

In [2]:
class WebScraper:
    
    def __init__(self):
        self.message = 'This is init'

    def extract_job_info(self,soup, job_info,role):
    #     job_info = {}
        temp = []
        jobdesc = ''
        mainpageURL = '' 
        count = len(job_info)
        for div in soup.find_all(name="div", attrs={"class":"jobsearch-SerpJobCard"}):

            for subdiv in div.find_all(name="div", attrs={"class":"sjcl"}):
                try:
                    s = subdiv.find(name="span", attrs={"class":"company"})
                    temp.append((s.text).strip()) #fetches company names (3)
                except:
                    temp.append("NA")

                try:
                    d = subdiv.find(name="div", attrs={"class":"recJobLoc"})
                    temp.append((d['data-rc-loc']).strip()) #fetches location (4)
                except:
                    temp.append("NA")
    #         print(div)

            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):

                try:
                    temp.append((a['title']).strip()) #fetches title (1)
                except:
                    temp.append("NA")

                try:
                    mainpageURL = str("https://www.indeed.com" + a["href"])
    #                 print(mainpageURL)
                    desc_page = requests.get(mainpageURL)
                    page_soup = BeautifulSoup(desc_page.text, "html.parser")
                    for dv in page_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
                        for l in dv.find_all("li"):
                            jobdesc = jobdesc +''+ l.text
    #                     print(jobdesc)
                        if jobdesc=='':
                            temp.append('NA')
                        else:
                            temp.append(jobdesc)      #fetches job description (2)
                except:
                    temp.append('NA')


            temp.append(role)

            job_info[count] = temp
            count = count + 1
            temp = []
            jobdesc = ''

        return(job_info)

    # job_info = extract_job_info(soup)

    def htmlParser(self,role):
        if role == 'software engineer':
            URL = "https://www.indeed.com/jobs?q=software+engineer&l=United+States"
            tempURL = URL
            num = 10
            job_info = {}
            for i in range(0,100):
                page = requests.get(URL)
    #             print(page)
                soup = BeautifulSoup(page.text, "html.parser")
    #             print(soup.prettify())
                URL = tempURL +'&start='+str(num)
    #             print(URL)
                num = num + 10
                job_info = self.extract_job_info(soup,job_info,role)

        elif role == 'data scientist':
            URL = "https://www.indeed.com/jobs?q=data+scientist&l=United+States"
            tempURL = URL
            num = 10
            job_info = {}
            for i in range(0,100):
                page = requests.get(URL)
    #             print(page)
                soup = BeautifulSoup(page.text, "html.parser")
    #             print(soup.prettify())
                URL = tempURL +'&start='+str(num)
    #             print(URL)
                num = num + 10
                job_info = self.extract_job_info(soup,job_info,role)

        elif role == 'data analyst':
            URL = "https://www.indeed.com/jobs?q=data+analyst&l=United+States"
            tempURL = URL
            num = 10
            job_info = {}
            for i in range(0,100):
                page = requests.get(URL)
    #             print(page)
                soup = BeautifulSoup(page.text, "html.parser")
    #             print(soup.prettify())
                URL = tempURL +'&start='+str(num)
    #             print(URL)
                num = num + 10
                job_info = self.extract_job_info(soup,job_info,role)

        elif role == 'machine learning engineer':
            URL = "https://www.indeed.com/jobs?q=machine+learning+engineer&l=United+States"
            tempURL = URL
            num = 10
            job_info = {}
            for i in range(0,100):
                page = requests.get(URL)
    #             print(page)
                soup = BeautifulSoup(page.text, "html.parser")
    #             print(soup.prettify())
                URL = tempURL +'&start='+str(num)
    #             print(URL)
                num = num + 10
                job_info = self.extract_job_info(soup,job_info,role)

        else:
            print('Enter a valid role')

        return job_info



class DumpData:
    
    def __init__(self):
        self.message = 'This is init'
            
    def uploadToS3(self,result):
        try:
            bucket = 'hiringtrendanalysis' # already created on S3
            csv_buffer = StringIO()
            result.to_csv(csv_buffer)
            s3_resource = boto3.resource('s3')
            s3_resource.Object(bucket, 'jobdesc.csv').put(Body=csv_buffer.getvalue())
        except ClientError as e:
            logging.error(e)
            return False
        return True
    
    def mergeDataframes(self,swe_info,da_info,ds_info,ml_info):
        swe_info_df = pd.DataFrame.from_dict(swe_info, orient='index', columns=['company','location','title','job_desc','role'])
        da_info_df = pd.DataFrame.from_dict(da_info, orient='index', columns=['company','location','title','job_desc','role'])
        ds_info_df = pd.DataFrame.from_dict(ds_info, orient='index', columns=['company','location','title','job_desc','role'])
        ml_info_df = pd.DataFrame.from_dict(ml_info, orient='index', columns=['company','location','title','job_desc','role'])
        
        frames = [swe_info_df,da_info_df,ds_info_df,ml_info_df]
        result = pd.concat(frames)
        return result

In [3]:
if __name__ == '__main__':
    
    webscraper = WebScraper()
    swe_info = webscraper.htmlParser('software engineer')
    da_info =  webscraper.htmlParser('data analyst')
    ds_info =  webscraper.htmlParser('data scientist')
    ml_info =  webscraper.htmlParser('machine learning engineer')
    print(len(swe_info))
    print(len(ds_info))
    print(len(da_info))
    print(len(ml_info))
    
    dmpdata = DumpData()
    result = dmpdata.mergeDataframes(swe_info,da_info,ds_info,ml_info)
    dmpdata.uploadToS3(result)
       

1864
1854
1692
1855


In [4]:
# swe_info = htmlParser('software engineer')
# print(len(swe_info))
# swe_info[15]

## scraping data analyst jobs

In [5]:
# da_info = htmlParser('data analyst')
# print(len(da_info))
# da_info[11]

## scraping data scientist jobs

In [6]:
# ds_info =  htmlParser('data scientist')
# print(len(ds_info))
# ds_info[11]

## scraping machine learning jobs

In [7]:
# ml_info =  htmlParser('machine learning engineer')
# print(len(ml_info))
# ml_info[11]

In [8]:
# swe_info_df = pd.DataFrame.from_dict(swe_info, orient='index', columns=['company','location','title','job_desc','role'])
# swe_info_df.head()

In [9]:
# da_info_df = pd.DataFrame.from_dict(da_info, orient='index', columns=['company','location','title','job_desc','role'])
# da_info_df.head()

In [10]:
# ds_info_df = pd.DataFrame.from_dict(ds_info, orient='index', columns=['company','location','title','job_desc','role'])
# ds_info_df.head()

In [11]:
# ml_info_df = pd.DataFrame.from_dict(ml_info, orient='index', columns=['company','location','title','job_desc','role'])
# ml_info_df.head()

In [13]:
# frames = [swe_info_df,da_info_df,ds_info_df,ml_info_df]
# result = pd.concat(frames)
# # result.head()
# # result.to_csv('temp.csv', index=False)
# uploadToS3(result)

In [14]:
# URL = "https://www.indeed.com/jobs?q=software+engineer&l=United+States"

# page = requests.get(URL)
# #             print(page)
# soup = BeautifulSoup(page.text, "html.parser")
# #             print(soup.prettify())

# jb = {}
# job_info = {}

# jb = extract_job_info(soup,job_info)

# jb[5]